# 优化器Optimizer
torch.optim 是 PyTorch 中的 优化器模块，用于更新神经网络中所有 可学习参数（带梯度的张量）

# 一、什么是 torch.optim？   
torch.optim 提供了一组优化器类（optimizers），这些类负责使用计算得到的梯度来更新模型的参数。   

# 二、PyTorch 训练流程中的位置 

```python
# 1. 前向传播
output = model(input)  # 输入数据通过模型，得到预测输出

# 2. 计算损失
loss = loss_fn(output, target)  # 使用损失函数计算预测值与真实值之间的误差

# 3. 清空梯度，否则会累加
optimizer.zero_grad()  # 清空之前计算的梯度，避免梯度累加

# 4. 反向传播，计算梯度
loss.backward()  # 通过反向传播计算每个参数的梯度

# 5. 使用优化器更新权重
optimizer.step()  # 根据梯度更新模型参数


```

# 三、常用优化器一览（torch.optim 中的类）     
![常用优化器](./images/Optim.png)

# 四、优化器常用参数
![优化器常用参数](./images/Optism2.png)

In [ ]:
# optim.SGD
#     params,                        要优化的参数，一般用model.parameters()
#     lr,                            学习率，控制每次参数更新的步长
#     momentum=0,                    动量，缓解震荡，加速收敛，常设为0.9
#     dampening=0,                   动量衰减，减少过快更新（一般不用）
#     weight_decay=0,                权重衰减，用于L2正则化，防止过拟合
#     nesterov=False,                是否使用Nesterov动量（更快收敛）
#     maximize=False,                若设为True，执行梯度上升（用于强化学习）
#     foreach=None,                  是否启用多张量操作（提高效率）
#     differentiable=False           是否启用二阶梯度（很少用）

In [1]:
import torch
import torchvision
from torch import nn
from collections import OrderedDict

In [2]:
class Tudui(nn.Module):
    def __init__(self):
        super(Tudui, self).__init__()
        # 使用 nn.Sequential 定义一个简单的卷积神经网络
        self.model1 = nn.Sequential(OrderedDict([
            ('conv1', nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, padding=2)),  # 第一层卷积
            ('pool1', nn.MaxPool2d(kernel_size=2)),  # 第一层池化
            ('conv2', nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, padding=2)),  # 第二层卷积
            ('pool2', nn.MaxPool2d(kernel_size=2)),  # 第二层池化
            ('conv3', nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=2)),  # 第三层卷积
            ('pool3', nn.MaxPool2d(kernel_size=2)),  # 第三层池化
            ('flatten', nn.Flatten()),  # 展平张量为一维
            ('fc1', nn.Linear(1024, 64)),  # 全连接层1，输入1024，输出64
            ('fc2', nn.Linear(64, 10))  # 全连接层2，输入64，输出10（分类数）
        ]))

    def forward(self, x):
        # 定义前向传播
        return self.model1(x)

In [3]:
# 加载 CIFAR10 数据集
dataset = torchvision.datasets.CIFAR10(
    "./data/CIFAR10/",  # 数据集存储路径
    train=False,  # 加载测试集
    transform=torchvision.transforms.ToTensor(),  # 将图像转换为张量
    download=True  # 如果数据集不存在，则下载
)
# 使用 DataLoader 加载数据集
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64)  # 每批次加载64张图片

Files already downloaded and verified


In [4]:
# 实例化模型
tudui = Tudui()
# 定义交叉熵损失函数
loss = nn.CrossEntropyLoss()
# 定义优化器，使用随机梯度下降（SGD）
optim = torch.optim.SGD(tudui.parameters(), lr=0.01)  # 学习率为0.01

# 训练循环
for epoch in range(5):  # 训练5个epoch
    running_loss = 0.0  # 初始化累计损失
    for data in dataloader:  # 遍历数据加载器
        imgs, targets = data  # 获取图像和对应的标签
        outputs = tudui(imgs)  # 前向传播，获取模型输出
        result_loss = loss(outputs, targets)  # 计算损失
        optim.zero_grad()  # 清空梯度
        result_loss.backward()  # 反向传播，计算梯度
        optim.step()  # 更新模型参数
        running_loss = running_loss + result_loss  # 累加损失
    print(running_loss)  # 打印每个epoch的累计损失

tensor(360.6461, grad_fn=<AddBackward0>)
tensor(356.7375, grad_fn=<AddBackward0>)
tensor(344.2848, grad_fn=<AddBackward0>)
tensor(324.0723, grad_fn=<AddBackward0>)
tensor(309.8381, grad_fn=<AddBackward0>)
